In [2]:
#!pip install plyer
#from plyer import notification
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import geckodriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
from datetime import datetime as date
import pandas as pd, numpy as np
import warnings as ws
ws.filterwarnings("ignore")
import re, os, time, glob
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
from selenium.webdriver.common.keys import Keys
import pickle
from typing import List
# Create directory if it doesn't exist
directory = "Fetched Data"
if not os.path.exists(directory):
    os.makedirs(directory)


def clean_str(string):
    string = string.replace('\xa0', " ").strip()
    string = string.replace('\n', "")
    string = string.replace('\t', "")
    return string.strip()

In [2]:
# Initialize Firefox with options
options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
geckodriver_autoinstaller.install()
firefox = webdriver.Firefox(options=options)

Firefox version:  117.0.1
Available geckodriver version:  v0.34.0


In [13]:
BASE_URL = 'https://dmec.moh.gov.vn/cong-khai-phan-loai-ttbyt'
firefox.get(BASE_URL)
WebDriverWait(firefox, timeout = 10)

<selenium.webdriver.support.wait.WebDriverWait (session="600c8895-02f4-4075-a8e2-5f93b37712ed")>

In [14]:
## Set per page device # to max (75 in this case)
res_per_page_div = firefox.find_element(By.CLASS_NAME, 'lfr-pagination-delta-selector')
res_per_page_div.click()


In [5]:
def extract_page_data(firefox):
    firefox.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    soup = BeautifulSoup(firefox.page_source)
    table = soup.find('table', {'class':'oep-table'})
    hrefs = [tr.find_all('td')[-1].find('a')['href'] for tr in table.find_all('tr')[1:]] 
    page_data = pd.read_html(str(table))[0]
    page_data['href'] = hrefs
    return page_data
    

In [16]:
page_counter = 0
collect_page_data = []
save_threshold = 10
while True:
    try:
        print(f'Page: {page_counter+1}')
        wait = WebDriverWait(firefox, 10)
        data = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'oep-table')))
        page_data = extract_page_data(firefox)
        collect_page_data.append(page_data)
        if len(collect_page_data) == save_threshold:
            threshold_save = pd.concat(collect_page_data)
            threshold_save.to_excel(f"Fetched Data\\Extracted_{page_counter} - {save_threshold}.xlsx")
            collect_page_data = []
            
        #go to next_page
        wait = WebDriverWait(firefox, 10)
        next_btn = wait.until(EC.element_to_be_clickable((By.XPATH, 
                                                          '/html/body/article/div/section/div/div/div/div/div/div/div/div/div/div/div/div/ul/li[3]/a')))
        firefox.execute_script("arguments[0].click();", next_btn)
        page_counter += 1
        time.sleep(5)
    except Exception as e:
        print(e)
        

Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:188:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:506:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:188:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:506:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:188:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:506:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

'NoneType' object is not subscriptable


KeyboardInterrupt: 

In [3]:
## Load scraped data for further processing
from glob import glob as g
import os
files = [pd.read_excel(file) for file in g(os.path.join("Fetched Data","*.xlsx"))]


In [24]:
from googletrans import Translator
translator = Translator()
from functools import lru_cache

@lru_cache(maxsize=None)
def translate_to_en(text):
    if pd.notna(text):
        translated_text = translator.translate(str(text), dest='en').text
        return translated_text
    else:
        return text

In [7]:
vn_mdr = pd.concat(files)
vn_mdr.drop_duplicates(inplace=True)
vn_mdr.shape

(115900, 10)

In [8]:
vn_mdr_en = vn_mdr.applymap(translate_vietnamese_to_english)

TypeError: expected string or bytes-like object

In [10]:
vn_mdr.columns

Index(['Unnamed: 0', 'STT', 'Tên trang thiết bị y tế',
       'Mức độ rủi rođược phân loại', 'Đơn vị thực hiện phân loại',
       'Số văn bản ban hành kết quảphân loại', 'Đơn vị yêu cầu phân loại',
       'Tình trạng', 'Thao tác', 'href'],
      dtype='object')

In [12]:
vn_mdr_en = vn_mdr.copy()
vn_mdr_en

,Unnamed: 0,STT,Tên trang thiết bị y tế,Mức độ rủi rođược phân loại,Đơn vị thực hiện phân loại,Số văn bản ban hành kết quảphân loại,Đơn vị yêu cầu phân loại,Tình trạng,Thao tác,href
0,0,74026,Máy loại bỏ paraffin,TTBYT Loại A,CÔNG TY CỔ PHẦN TƯ VẤN ĐẦU TƯ VÀ PHÁT TRIỂN CÔ...,42721CN/190000014/PCBPL-BYT,Công ty TNHH Xuất Nhập Khẩu Vật Tư Khoa Học Qu...,Còn hiệu lực 02/08/2021,NaN,/documents/10182/20903018/upload_00018221_1627...
1,1,74027,Máy loại bỏ sắc tố da bằng công nghệ Picolaser,TTBYT Loại B,CÔNG TY TNHH THIẾT BỊ THẨM MỸ IDM VIỆT NAM,02/2023/PL-IDM,NaN,Còn hiệu lực 23/01/2024,NaN,/documents/10182/51397734/upload_00017585_1705...
2,2,74028,Máy lọc gan MARS Monitor 1TC,TTBYT Loại C,CÔNG TY CỔ PHẦN THIẾT BỊ Y TẾ THIÊN ÂN,1905/170000074/PCBPL-BYT,Công ty TNHH Baxter Việt Nam Healthcare,Còn hiệu lực 29/02/2020,NaN,/documents/10182/14150513/upload_00292641_1582...
3,3,74029,Máy lọc gan tích hợp với máy lọc máu liên tục,TTBYT Loại C,CÔNG TY CỔ PHẦN THIẾT BỊ Y TẾ THIÊN ÂN,2167 /170000074/PCBPL-BYT,Công ty TNHH Baxter Việt Nam Healthcare,Còn hiệu lực 02/05/2021,NaN,/documents/10182/19433099/upload_00006210_1619...
4,4,74030,Máy lọc gan tích hợp với máy lọc máu liên tục,TTBYT Loại C,CÔNG TY CỔ PHẦN THIẾT BỊ Y TẾ THIÊN ÂN,2167/170000074/PCBPL-BYT,VPĐD Baxter Healthcare (Asia) Pte Ltd tại TP H...,Đã thu hồi 02/05/2021,NaN,/documents/10182/19433099/upload_00005019_1619...
...,...,...,...,...,...,...,...,...,...,...
745,70,74021,Máy li tâm và phụ kiện,TTBYT Loại A,CÔNG TY TNHH THIẾT BỊ Y TẾ MINH KHOA,96/170000083/PCBPL-BYT,CTY TNHH THIẾT BỊ Y TẾ MINH KHOA,Còn hiệu lực 17/07/2019,NaN,/documents/10182/10699366/upload_00063476_1563...
746,71,74022,Máy li tâm và phụ kiện tiêu chuẩn đi kèm,TTBYT Loại B,CÔNG TY TNHH CÔNG NGHỆ ADJ VIỆT NAM,20180169-ADJVINA/170000008/PCBPL-BYT,CÔNG TY CỔ PHẦN MEDCOMTECH,Còn hiệu lực 14/01/2021,NaN,/documents/10182/18145075/upload_00055369_1610...
747,72,74023,Máy Lipomax Plus 15.7,TTBYT Loại B,CÔNG TY TNHH XUÂN VY,047-XV/2017/170000024/PCBPL-BYT,CÔNG TY TNHH THƯƠNG MẠI VÀ DỊCH VỤ CHÂU ĐẠI DƯƠNG,Còn hiệu lực 07/04/2020,NaN,/documents/10182/14674465/upload_00247178_1586...
748,73,74024,Máy loại bỏ mô,TTBYT Loại A,CÔNG TY TNHH RICHARD WOLF VIỆT NAM,85/RWVN1223,NaN,Còn hiệu lực 03/01/2024,NaN,/documents/10182/50838896/upload_00028208_1704...


In [ ]:
vn_mdr_en['Tên trang thiết bị y tế'] = vn_mdr_en['Tên trang thiết bị y tế'].map(lambda x: translate_vietnamese_to_english(x))

In [ ]:
vn_mdr_en['Mức độ rủi rođược phân loại'] = vn_mdr_en['Mức độ rủi rođược phân loại'].map(lambda x: translate_vietnamese_to_english(x))
print("Done")

In [ ]:
vn_mdr_en['Đơn vị thực hiện phân loại'] = vn_mdr_en['Đơn vị thực hiện phân loại'].map(lambda x: translate_vietnamese_to_english(x))
print("Done")

In [ ]:
vn_mdr_en['Đơn vị yêu cầu phân loại'] = vn_mdr_en['Đơn vị yêu cầu phân loại'].map(lambda x: translate_vietnamese_to_english(x))
print("Done")

In [ ]:
vn_mdr_en['Tình trạng'] = vn_mdr_en['Tình trạng'].map(lambda x: translate_vietnamese_to_english(x))
print("Done")

In [ ]:
print("ALL DONE")

In [ ]:
from plyer import notification
notification.notify(
    title='Gateway Alerts',
    message='VN MDR has been translated to English!',
    app_name='Gateway Data Miner',
    app_icon='C:\\Users\\AbhishekSrivastava\\The Gateway\\gateway_process.ico'
)

In [31]:
len(all_rows)

75

In [19]:
rows

[<tr>
 <th style="width:3%;">STT</th>
 <th style="width:20%;">Tên trang thiết bị y tế</th>
 <th style="width:12%;">Mức độ rủi ro<br/>được phân loại</th>
 <th>Đơn vị thực hiện phân loại</th>
 <th style="width:12%;">Số văn bản <br/>ban hành kết quả<br/>phân loại</th>
 <th>Đơn vị yêu cầu phân loại</th>
 <th style="width:10%;">Tình trạng</th>
 <th style="width:5%;">Thao tác</th>
 </tr>,
 <tr>
 <td style="width:3%; text-align: center;">1</td>
 <td style="width:20%;text-align: left;">	 Kim luồn tĩnh mạch có cánh, có cổng tiêm thuốc, chất liệu ETFE, cản quang   Nipro Wing Cath IV cannula with injection port &amp; wing ETFE radiopaque catheter</td>
 <td style="width:12%;text-align: center;">TTBYT Loại C</td>
 <td>TRUNG TÂM NGHIÊN CỨU VÀ TƯ VẤN KỸ THUẬT THIẾT BỊ Y TẾ</td>
 <td style="width:12%;text-align: center;">
 				01/1709/MERAT-2019
 				
 			</td>
 <td>Công ty TNHH y tế Việt Tiến</td>
 <td style="width:10%;text-align: center;">
 <span>Còn hiệu lực</span><br/>
 <span style="font-

In [4]:
import re
from transformers import MarianMTModel, MarianTokenizer

# Load the model and tokenizer
model_name = 'Helsinki-NLP/opus-mt-vi-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def translate_vietnamese_to_english(text):
    # Separate text and date
    match = re.match(r"(.*?)(\d{2}-\d{2}-\d{4})", text)
    if match:
        text_to_translate, date = match.groups()
    else:
        text_to_translate, date = text, ''

    # Tokenize the text
    batch = tokenizer([text_to_translate], return_tensors="pt", padding=True)

    # Generate translation
    translated = model.generate(**batch)

    # Decode the translated text
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)

    # Combine translated text with date
    return f"{translated_text} {date}".strip()

# Example
example_text = "Hết hạn 12-03-2023"  # Example Vietnamese text with a date
translation = translate_vietnamese_to_english(example_text)
print(translation)


Timeout 12-03-2023
